In [1]:
from main import *

Generator of the curve : (74366762915394286921614637097245469780 : 8044611866574517032690388593663626080 : 1) of order 340282366920938463488226353707043809157


In [2]:
keys = [np.random.randint(0, 2**64, dtype=np.uint64) for _ in range(2)]
print("Keys:")
print(f"{keys[0]:x}")
print(f"{keys[1]:x}")


Keys:
b3f018d080b0c97c
a340885a44905c74


In [3]:
x = np.uint16(0x3f7b)

print(f"{sbox_inv(sbox(x)):x}")
print(f"{x:x}")

3f7b
3f7b


/Users/neosapien/Development/exam-sg8/main.py:35: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_sbox # 50 / 4 = 12.5
/Users/neosapien/Development/exam-sg8/main.py:37: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_sbox # 12.5
/Users/neosapien/Development/exam-sg8/main.py:43: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_inv
/Users/neosapien/Development/exam-sg8/main.py:46: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_inv


In [4]:
message = int(''.join([str(np.random.randint(0, 2**64, dtype=np.uint64)) for _ in range(16)]))
print("Message:", f"{message:x}")

aux, cipher, tag = encrypt_and_authenticate(keys, message)
print("Cipher:", f"{cipher:x}")

verified, decrypted = decrypt_and_verify(keys, cipher, tag, aux)
print("Verified:", verified)
if verified:
    print("Decrypted:", f"{decrypted:x}")


Message: 25c928d4f2200a34ed21083827836db5b9cff50c90f496350766da84191e28765fcbbdf0f570929953d42b0160e83eab7a7e8edc80c7d0d2061de1b8b60b7cfe5a07459ec0ae576eaf5f9a21bcf7c1f8cca24d711a04e3d7205049b4dc43a80f71265643ff76e2b3dbc70adc4ad5a3318d3b9fe7d973f81641baa685c4c3e3cba8c
Cipher: f578f192ae05f967f85d846620646a355bb54e7775bf059b5c5c885a9c56d80fdd380c66eeb5468e231fce8617287e611980183298b4b65e107e823eb403cdeb4a6a952e875d4302597963838b4a0657ed84497fa8eda13e3574a0d651960efd19030145c31d17cb0918aae5e031359cf574e68d3c088e1ec99cd5d00b54b44ac7aefbae8ec5dc3
Verified: True
Decrypted: 25c928d4f2200a34ed21083827836db5b9cff50c90f496350766da84191e28765fcbbdf0f570929953d42b0160e83eab7a7e8edc80c7d0d2061de1b8b60b7cfe5a07459ec0ae576eaf5f9a21bcf7c1f8cca24d711a04e3d7205049b4dc43a80f71265643ff76e2b3dbc70adc4ad5a3318d3b9fe7d973f81641baa685c4c3e3cba8c


In [5]:
priv_k, pub_k = signature_keygen()

print(f"Private key: {priv_k}")
print(f"Public key: {pub_k}")

signature = sign(priv_k, message)
print(f"Signature: {signature}")

verified = verify(pub_k, signature, message)
print(f"Verified: {verified}")

Private key: 261079820640636513735762951947329265777
Public key: (164328089636669710544227988296625490612 : 138187611956126640940991475514053106065 : 1)
Signature: (56879374649909198412625442457126187540, 336010560294298093931644566292196463357)
Verified: True


In [6]:
simulate_akex()

Simulating AKEX
A: Private key: 107457198648652796125904606285717545379
A: Public key: (195570290001725014523938924531588259777 : 205365772810327924164725202238786286762 : 1)
B: Private key: 3255124564454736250553664640405676175
B: Public key: (209513939205357082390611952080588287953 : 284606625835356304922932235749210892440 : 1)
Alice sends signed message (A)
Bob sends signed message (B)
Alice checks Bob is who he says he is before checking the key
Alice checks the key
[7719556496474859749, 5352144588021282289]
[7719556496474859749, 5352144588021282289]


In [7]:
def differential_attack_round(round_func, num_tests=10000, samples_per_test=10000):
    """Perform differential cryptanalysis on a single round function"""
    best_differentials = []
    
    for _ in range(num_tests):
        # Random input difference
        alpha = np.random.randint(0, 2**64, dtype=np.uint64)
        beta_counter = {}
        
        # Test many input pairs with this difference
        for _ in range(samples_per_test):
            m = np.random.randint(0, 2**64, dtype=np.uint64)
            m_prime = m ^ alpha
            beta = round_func(m) ^ round_func(m_prime)
            beta_counter[beta] = beta_counter.get(beta, 0) + 1
            
        # Find most common output difference
        most_common_beta = max(beta_counter.items(), key=lambda x: x[1])
        prob = most_common_beta[1] / samples_per_test
        best_differentials.append((alpha, most_common_beta[0], prob))
    
    # Sort by probability
    best_differentials.sort(key=lambda x: x[2], reverse=True)
    return best_differentials

def linear_attack_round(round_func, num_tests=10000, samples_per_test=10000):
    """Perform linear cryptanalysis on a single round function"""
    best_linear_pairs = []
    
    def parity(x, y):
        return bin(x & y).count('1') % 2
    
    for _ in range(num_tests):
        # Random input/output masks
        a = np.random.randint(0, 2**64, dtype=np.uint64)
        b = np.random.randint(0, 2**64, dtype=np.uint64)
        
        # Calculate bias
        lat_approx = 0
        for _ in range(samples_per_test):
            x = np.random.randint(0, 2**64, dtype=np.uint64)
            ax_parity = parity(a, x)
            bs_parity = parity(b, round_func(x))
            
            if ax_parity == bs_parity:
                lat_approx += 1
            else:
                lat_approx -= 1
                
        bias = lat_approx / (2.0 * samples_per_test)
        prob = 0.5 + abs(bias)
        
        if abs(bias) > 0.01:  # Only keep significant biases
            best_linear_pairs.append((a, b, prob))
    
    # Sort by probability
    best_linear_pairs.sort(key=lambda x: x[2], reverse=True)
    return best_linear_pairs

def test_multiple_rounds(max_rounds=8):
    """Test differential and linear attacks for multiple rounds"""
    test_key = np.uint64(0x1234567890ABCDEF)
    
    print("Testing attacks for multiple rounds...")
    print("\nDifferential Attack Results:")
    print("Rounds | Best Probability")
    print("-" * 30)
    
    for n in range(1, max_rounds + 1):
        def n_rounds(x):
            result = x
            for _ in range(n):
                result = block_encryption_round(test_key, result)
            return result
        
        diff_results = differential_attack_round(n_rounds, num_tests=100, samples_per_test=100)
        best_diff_prob = diff_results[0][2] if diff_results else 0
        print(f"{n:6d} | {best_diff_prob:.6f}")
    
    print("\nLinear Attack Results:")
    print("Rounds | Best Probability")
    print("-" * 30)
    
    for n in range(1, max_rounds + 1):
        def n_rounds(x):
            result = x
            for _ in range(n):
                result = block_encryption_round(test_key, result)
            return result
        
        linear_results = linear_attack_round(n_rounds, num_tests=100, samples_per_test=100)
        best_linear_prob = linear_results[0][2] if linear_results else 0.5
        print(f"{n:6d} | {best_linear_prob:.6f}")

# Run the tests
test_multiple_rounds()


Testing attacks for multiple rounds...

Differential Attack Results:
Rounds | Best Probability
------------------------------
     1 | 0.010000
     2 | 0.010000
     3 | 0.010000


KeyboardInterrupt: 

In [ ]:
from main import *


# A generates a public-private key pair
priv_key_A, pub_key_A = signature_keygen()

# B does the same
priv_key_B, pub_key_B = signature_keygen()

# A signs a message
message = np.uint64(0x1234567890ABCDEF)
signature = sign(priv_key_A, message)

# A and B exchange their public keys
internal_secret_A, msg1 = akex_init(priv_key_A, pub_key_A)
internal_secret_B, msg2 = akex_init(priv_key_B, pub_key_B)

# A and B verify each other's identity
shared_keys_a = akex_final(pub_key_B, internal_secret_A, msg2)
shared_keys_b = akex_final(pub_key_A, internal_secret_B, msg1)

shared_keys_a = [int(s) for s in shared_keys_a]
shared_keys_b = [int(s) for s in shared_keys_b]

# A uses the shared key to encrypt a message and signature to send to B
message = np.uint64(0x1234567890ABCDEF)
encrypted_message, num_blocks = block_encrypt(shared_keys_a[0], message)
encrypted_signature_r, num_blocks_signature_r = block_encrypt(shared_keys_a[1], signature[0])
encrypted_signature_s, num_blocks_signature_s = block_encrypt(shared_keys_a[1], signature[1])

# A and B verify the message
verified_message = block_decrypt(shared_keys_b[0], encrypted_message, num_blocks)
verified_signature_r = block_decrypt(shared_keys_b[1], encrypted_signature_r, num_blocks_signature_r)
verified_signature_s = block_decrypt(shared_keys_b[1], encrypted_signature_s, num_blocks_signature_s)
verified_signature = (verified_signature_r, verified_signature_s)

# A and B verify the signature
verified = verify(pub_key_B, verified_signature, message)


In [1]:
from main import *

simulate_complete_protocol()
simulate_signature()
simulate_akex()
simulate_encryption()

Generator of the curve : (74366762915394286921614637097245469780 : 8044611866574517032690388593663626080 : 1) of order 340282366920938463488226353707043809157
Simulating complete protocol
A generates a public-private key pair
B does the same
A signs a message
A and B exchange their public keys
A and B verify each other's identity
A uses the shared key to encrypt a message and signature to send to B
B decrypts and verifies the message
B decrypts and verifies the signature
Verified: True
Simulating signature
Verified: True
Simulating AKEX
A: Private key: 119458704371673837036014741144857472101
A: Public key: (40230606864096711387233737677844227990 : 230083062250097558453985189737763126852 : 1)
B: Private key: 63561771952311148974401684931478301768
B: Public key: (21757964953305448609793968406916206267 : 277293703457790338560787110881666271555 : 1)
Alice sends signed message (A)
Bob sends signed message (B)
Alice checks Bob is who he says he is before checking the key
Alice checks the key

/Users/neosapien/Development/exam-sg8/main.py:35: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_sbox # 50 / 4 = 12.5
/Users/neosapien/Development/exam-sg8/main.py:37: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_sbox # 12.5
/Users/neosapien/Development/exam-sg8/main.py:43: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_inv
/Users/neosapien/Development/exam-sg8/main.py:46: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_inv
